# Baseline - Train

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

from PIL import Image
Image.MAX_IMAGE_PIXELS = None

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.utils import make_grid
import torchvision.datasets as datasets
import torchvision.transforms as T

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split

from sklearn.model_selection import KFold

import albumentations as A

from pycocotools.coco import COCO

In [3]:
import torch

print('cuda') if torch.cuda.is_available() else print('cpu')

cuda


In [4]:
# import tensorflow as tf

# print(tf.config.list_physical_devices('GPU'))

In [5]:
"""
# torch
conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia

# albumentations
pip install -U albumentations
# libGL.so.1: cannot open shared object file: No such file or directory
apt-get update && apt-get install libgl1

# PIL
pip install pillow

# openCV
pip install opencv-python

# sklearn
pip install scikit-learn

# ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant'
pip install chardet

# numpy pandas matplolib seaborn...
"""

"\n# torch\nconda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia\n\n# albumentations\npip install -U albumentations\n# libGL.so.1: cannot open shared object file: No such file or directory\napt-get update && apt-get install libgl1\n\n# PIL\npip install pillow\n\n# openCV\npip install opencv-python\n\n# sklearn\npip install scikit-learn\n\n# ImportError: cannot import name 'COMMON_SAFE_ASCII_CHARACTERS' from 'charset_normalizer.constant'\npip install chardet\n\n# numpy pandas matplolib seaborn...\n"

In [6]:
cwd = "/USER/mnc/fish_copy/data/"

train_dir = cwd + "train/"
test_dir  = cwd + "test/"

train_list = os.listdir(train_dir)
test_list  = os.listdir(test_dir)

print("Original file len")
print("train : 104876, test : 44947")
print("Copied file len")
print(f"train : {len(train_list)}, test : {len(test_list)}")

train_list = sorted(train_list, key= lambda x: int(x[6:-4]))
test_list  = sorted(test_list, key= lambda x: int(x[5:-4]))

Original file len
train : 104876, test : 44947
Copied file len
train : 104875, test : 44946


In [7]:
ann_json    = cwd + 'labels/train.json'
answer_json = cwd + 'labels/answer_sample.json'

coco = COCO(ann_json)
coco_answer = COCO(answer_json)

loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [ ]:
# Category ID check
coco.getCatIds()

# find annotation ID of image_id
coco.getAnnIds(imgIds=1)
# find annotation ID of category_id
coco.getAnnIds(catIds=1)

# find Image ID of category_id
coco.getImgIds(catIds=1)

# Specific data of category_id
coco.loadCats(1)

# All specific annotation dict data of annotation_id
coco.loadAnns(1)

# All specific image dict data of image_id
coco.loadImgs(1)

In [ ]:
# category 정보 추출
categories = coco.getCatIds()

# 해당 category에 해당하는 image_id 다 찾기
for i in categories:
    globals()[f"category{i}_imgid"] = coco.getImgIds(catIds=i)

# Created list category0_imgid ~ category7imgid

# image_id를 이용해서 file_name찾기
for i in categories:
    globals()[f"category{i}_imgname"] = []
    for j in eval(f"category{i}_imgid"):
        eval(f"category{i}_imgname").append(coco.loadImgs(j)[0]['file_name'])

# Created list category0_imgname ~ category7_imgname

"""
# image_id로 annotation_id 찾기
여기서 이미지 아이디를 기반으로 어노테이션 아이디를 찾고

# annotation_id로 해당 annotation정보 찾기
찾은 어노테이션 아이디를 기반으로 bbox 정보 찾기
"""

In [ ]:
category0_imgid

In [23]:
train_label = [0 for i in range(len(train_list))]
len(train_list), len(train_label)

104875

In [49]:
coco.loadImgs()

[]

In [40]:
coco.loadImgs(5543)

[{'id': 5543,
  'width': 640,
  'height': 480,
  'file_name': 'train_3865.png',
  'license': 0,
  'flickr_url': '',
  'coco_url': '',
  'date_captured': 0}]

In [48]:
coco.loadAnns(5543)

[{'id': 5543,
  'image_id': 98263,
  'category_id': 2,
  'segmentation': [],
  'area': 11502.345400000006,
  'bbox': [351.64, 305.7, 139.49, 82.46],
  'iscrowd': 0,
  'attributes': {'occluded': False, 'rotation': 0.0}}]

[]

In [16]:
whole_image_ids = coco.getImgIds()
image_ids = []
no_anno_ids = []

# to remove not annotated image idx
for idx in whole_image_ids:
    annotations_ids = coco.getAnnIds(imgIds=idx, iscrowd=False)
    if len(annotations_ids) == 0:
        no_anno_ids.append(idx)
    else:
        image_ids.append(idx)


train_files = []
for id in image_ids:
    train_file = coco.loadImgs(id)[0]['file_name']
    train_files.append(train_file)

print("train_files: ", len(train_files))

train_files:  5561


In [17]:
image_ids[:5], train_files[:5], len(no_anno_ids)

([1, 283, 284, 292, 297],
 ['train_0.png',
  'train_185.png',
  'train_186.png',
  'train_192.png',
  'train_194.png'],
 99314)

In [27]:
import shutil

output_path = '/USER/mnc/fish/data/train_fish'

def load_images_from_folder(folder, train_files, output_path):
    count = 0
    for filename in tqdm(train_files):
        source = os.path.join(folder, filename)
        destination = f"{output_path}/{filename[:-4]}.jpg"
        print(source)
        print(destination)
        break
        
        try:
            shutil.copy(source, destination)
            print("File copied successfully.")

        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        count += 1

load_images_from_folder(folder='/USER/mnc/fish/data/train', train_files=train_files, output_path=output_path)

  0%|          | 0/5561 [00:00<?, ?it/s]

/USER/mnc/fish/data/train/train_0.png
/USER/mnc/fish/data/train_fish/train_0.jpg


In [25]:
coco.getCatIds(catNms='농어')

[0]

In [ ]:
image_id, file_name

In [7]:
category, bbox = coco.loadAnns(coco.getAnnIds(1))[0]['category_id'], coco.loadAnns(coco.getAnnIds(1))[0]['bbox']

category, bbox

(3, [37.98, 209.94, 369.19, 110.89])

In [9]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, clear_output

coco_class = ['농어', '베스', '숭어', '강준치', '블루길', '잉어', '붕어', '누치']

for num in range(1000):
    img = Image.open(cwd + "train/" + train_files[num])
    category, bbox = coco.loadAnns(coco.getAnnIds(image_ids[num]))[0]['category_id'], coco.loadAnns(coco.getAnnIds(image_ids[num]))[0]['bbox']

    draw = ImageDraw.Draw(img)
    font = ImageFont.truetype("root/anaconda3/envs/fish/lib/python3.10/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf", 30)
    draw.text((250, 5), coco_class[category], fill='red', font=font)


    # COCO annotation에서 (x, y, width, height) 형태를 (x, y, x+width, y+height)로 변환
    x, y, width, height = bbox
    x, y, x2, y2 = x, y, x + width, y + height

    # bounding box를 시각화
    # rect = patches.Rectangle((x, y), width, height, linewidth=2, edgecolor="r", facecolor="none", label=f"Category {category}")
    # plt.gca().add_patch(rect)
    draw.rectangle([x, y, x2, y2], outline='red')

    display(img)

    # 사용자의 입력을 기다립니다.
    user_input = input("Press Enter to open the next image, or 'q' to quit...")

    # 이미지를 닫습니다.
    clear_output()

    # 사용자가 'q'를 누르면 루프를 종료합니다.
    if user_input.lower() == 'q':
        break

# DataLoader

In [33]:
class FishDataLoader(Dataset):
    def __init__(self, root_dir="/USER/mnc/fish/data/", train_list=None, train_label=None, train=True, augmentations=None):
        super().__init__()
        # self.train = train
        self.transforms    = T.ToTensor()
        self.root_dir      = root_dir
        self.train         = True if train else False
        self.leaf_data_dir = 'train/' if train else 'test/'
        self.json_name     = 'train.json' if train else None
        
        self.train_list  = train_list
        self.train_label = train_label
        
    def __getitem__(self, index):
        img = Image.open(self.root_dir + self.leaf_data_dir + self.train_list[index]).convert('RGB')
        img = self.transforms(img)
        if self.train:
            label = self.train_label[index]
            label = torch.tensor(label, dtype=torch.int8)
        else:
            label = None
        return img, label
    
    def __len__(self):
        return len(self.train_list)

In [34]:
data_loader = FishDataLoader(train_list=train_list, train_label=train_label)

In [35]:
next(iter(data_loader))

(tensor([[[0.2588, 0.2588, 0.2784,  ..., 0.3373, 0.3373, 0.3373],
          [0.2588, 0.2588, 0.2784,  ..., 0.3333, 0.3333, 0.3333],
          [0.2510, 0.2510, 0.2941,  ..., 0.3333, 0.3333, 0.3333],
          ...,
          [0.0863, 0.0863, 0.0863,  ..., 0.0235, 0.0157, 0.0235],
          [0.0902, 0.0902, 0.0902,  ..., 0.0000, 0.0000, 0.0000],
          [0.0902, 0.0902, 0.0902,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.2863, 0.2863, 0.2745,  ..., 0.3451, 0.3451, 0.3451],
          [0.2863, 0.2863, 0.2745,  ..., 0.3412, 0.3412, 0.3412],
          [0.2902, 0.2902, 0.2627,  ..., 0.3412, 0.3412, 0.3412],
          ...,
          [0.0941, 0.0941, 0.0941,  ..., 0.0314, 0.0235, 0.0314],
          [0.0980, 0.0980, 0.0980,  ..., 0.0039, 0.0000, 0.0039],
          [0.0980, 0.0980, 0.0980,  ..., 0.0000, 0.0000, 0.0039]],
 
         [[0.2667, 0.2667, 0.2863,  ..., 0.3333, 0.3333, 0.3333],
          [0.2667, 0.2667, 0.2863,  ..., 0.3294, 0.3294, 0.3294],
          [0.2588, 0.2588, 0.2863,  ...,